<a href="https://colab.research.google.com/github/LucasG99/modelo-fraude/blob/main/C%C3%B3pia_de_Ponderada_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Descrição da ponderada:**

Objetivo: Otimizar um modelo de rede neural pré-treinado para detecção de fraudes em cartões de crédito. Aplicar técnicas avançadas de ajuste fino de hiperparâmetros, como grid search e random search, com o objetivo de aprimorar as métricas de desempenho do modelo, incluindo precisão, recall, F1-score e AUC-ROC. A atividade também exige uma comparação entre o modelo otimizado e o modelo original, permitindo avaliar o impacto das modificações nos hiperparâmetros sobre o desempenho geral.

In [24]:
%pip install gdown
import gdown

In [25]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=d9459599-8de4-48e5-a4fb-47b5b14c122a
To: /content/dataset.csv
100%|██████████| 151M/151M [00:04<00:00, 33.5MB/s]


'dataset.csv'

Daqui pra frente é o pré processamento ...

In [26]:
df.head(), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

(   Time        V1        V2        V3        V4        V5        V6        V7  \
 0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
 1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
 2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
 3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
 4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   
 
          V8        V9  ...       V21       V22       V23       V24       V25  \
 0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
 1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
 2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
 3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
 4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   
 
         V26      

In [27]:
df["Class"].value_counts(normalize=True).rename("proportion")

,proportion
Class,
0,0.998273
1,0.001727


In [28]:
import numpy as np, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
tf.random.set_seed(42)

X = df.drop("Class", axis=1).values
y = df["Class"].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

neg, pos = np.bincount(y_train)
total = neg + pos
class_weights = {0: (1/neg)*(total/2.0), 1: (1/pos)*(total/2.0)}


In [29]:
from tensorflow import keras
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score

model = keras.Sequential([
    keras.Input(shape=(X_train_s.shape[1],)),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss="binary_crossentropy", metrics=["accuracy"])

model.fit(X_train_s, y_train, epochs=10, batch_size=2048,
          validation_split=0.2, class_weight=class_weights, verbose=0)

y_proba = model.predict(X_test_s, verbose=0)
y_pred = (y_proba >= 0.5).astype(int)

print(classification_report(y_test, y_pred, digits=4))
print("AUC-ROC:", roc_auc_score(y_test, y_proba))

baseline_result = {
    "accuracy": accuracy_score(y_test, y_pred),
    "precision_1": classification_report(y_test, y_pred, output_dict=True)["1"]["precision"],
    "recall_1": classification_report(y_test, y_pred, output_dict=True)["1"]["recall"],
    "f1_1": classification_report(y_test, y_pred, output_dict=True)["1"]["f1-score"],
    "auc": roc_auc_score(y_test, y_proba)
}
baseline_result


              precision    recall  f1-score   support

           0     0.9998    0.9839    0.9918     85295
           1     0.0860    0.8716    0.1566       148

    accuracy                         0.9837     85443
   macro avg     0.5429    0.9278    0.5742     85443
weighted avg     0.9982    0.9837    0.9903     85443

AUC-ROC: 0.9676143448096669


{'accuracy': 0.9837318446215606,
 'precision_1': 0.086,
 'recall_1': 0.8716216216216216,
 'f1_1': 0.15655339805825244,
 'auc': np.float64(0.9676143448096669)}

In [30]:
def train_and_evaluate(n_units=32, n_hidden=1, dropout=0.0, lr=1e-3, epochs=10, batch_size=1024):
    m = keras.Sequential()
    m.add(keras.Input(shape=(X_train_s.shape[1],)))
    for _ in range(n_hidden):
        m.add(keras.layers.Dense(n_units, activation="relu"))
        if dropout > 0:
            m.add(keras.layers.Dropout(dropout))
    m.add(keras.layers.Dense(1, activation="sigmoid"))
    m.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
              loss="binary_crossentropy", metrics=["accuracy"])
    m.fit(X_train_s, y_train, epochs=epochs, batch_size=batch_size,
          validation_split=0.2, class_weight=class_weights, verbose=0)
    p = m.predict(X_test_s, verbose=0)
    yhat = (p >= 0.5).astype(int)
    rep = classification_report(y_test, yhat, output_dict=True)
    return {
        "n_units": n_units,
        "n_hidden": n_hidden,
        "dropout": dropout,
        "lr": lr,
        "epochs": epochs,
        "batch_size": batch_size,
        "accuracy": accuracy_score(y_test, yhat),
        "precision_1": rep["1"]["precision"],
        "recall_1": rep["1"]["recall"],
        "f1_1": rep["1"]["f1-score"],
        "auc": roc_auc_score(y_test, p)
    }

import pandas as pd

results = []
results.append({**{"label":"Baseline"}, **baseline_result})
results.append(train_and_evaluate(n_units=32, n_hidden=1, dropout=0.0, lr=1e-3, epochs=10, batch_size=1024))
results.append(train_and_evaluate(n_units=64, n_hidden=2, dropout=0.2, lr=1e-3, epochs=20, batch_size=1024))
results.append(train_and_evaluate(n_units=128, n_hidden=2, dropout=0.3, lr=5e-4, epochs=20, batch_size=1024))
results.append(train_and_evaluate(n_units=256, n_hidden=3, dropout=0.4, lr=1e-4, epochs=30, batch_size=1024))

df_exp = pd.DataFrame(results)
df_exp


,label,accuracy,precision_1,recall_1,f1_1,auc,n_units,n_hidden,dropout,lr,epochs,batch_size
0,Baseline,0.983732,0.086000,0.871622,0.156553,0.967614,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.983486,0.082065,0.837838,0.149488,0.954251,32.0,1.0,0.0,0.0010,10.0,1024.0
2,NaN,0.991936,0.158890,0.851351,0.267800,0.963450,64.0,2.0,0.2,0.0010,20.0,1024.0
3,NaN,0.992580,0.172507,0.864865,0.287640,0.964780,128.0,2.0,0.3,0.0005,20.0,1024.0
4,NaN,0.986904,0.104319,0.864865,0.186182,0.962445,256.0,3.0,0.4,0.0001,30.0,1024.0


In [31]:
param_space = {
    "n_units": [32, 64, 128, 256],
    "n_hidden": [1, 2, 3],
    "dropout": [0.0, 0.2, 0.3, 0.4],
    "lr": [1e-3, 5e-4, 1e-4],
    "epochs": [10, 20, 30],
    "batch_size": [512, 1024, 2048]
}

rng = np.random.RandomState(7)
rand_results = []
for _ in range(15):
    cfg = {
        "n_units": rng.choice(param_space["n_units"]),
        "n_hidden": rng.choice(param_space["n_hidden"]),
        "dropout": rng.choice(param_space["dropout"]),
        "lr": rng.choice(param_space["lr"]),
        "epochs": int(rng.choice(param_space["epochs"])),
        "batch_size": int(rng.choice(param_space["batch_size"]))
    }
    rand_results.append(train_and_evaluate(**cfg))

df_rand = pd.DataFrame(rand_results)
best_f1_row = df_rand.sort_values("f1_1", ascending=False).iloc[0]
best_auc_row = df_rand.sort_values("auc", ascending=False).iloc[0]

df_top_f1 = df_rand.sort_values("f1_1", ascending=False).head(5)
df_top_auc = df_rand.sort_values("auc", ascending=False).head(5)

best_f1_row, best_auc_row, df_top_f1, df_top_auc


(n_units          32.000000
 n_hidden          3.000000
 dropout           0.000000
 lr                0.000500
 epochs           30.000000
 batch_size     2048.000000
 accuracy          0.995248
 precision_1       0.243028
 recall_1          0.824324
 f1_1              0.375385
 auc               0.943476
 Name: 5, dtype: float64,
 n_units         256.000000
 n_hidden          1.000000
 dropout           0.200000
 lr                0.000100
 epochs           10.000000
 batch_size     1024.000000
 accuracy          0.986646
 precision_1       0.101205
 recall_1          0.851351
 f1_1              0.180905
 auc               0.976551
 Name: 0, dtype: float64,
     n_units  n_hidden  dropout      lr  epochs  batch_size  accuracy  \
 5        32         3      0.0  0.0005      30        2048  0.995248   
 2        32         3      0.0  0.0010      10         512  0.995155   
 9       256         1      0.2  0.0010      20        1024  0.994183   
 13       64         2      0.2  0.0010 

In [32]:
print("Melhor por F1_1:", best_f1_row[["n_units","n_hidden","dropout","lr","epochs","batch_size","f1_1","auc"]].to_dict())
print("Melhor por AUC:", best_auc_row[["n_units","n_hidden","dropout","lr","epochs","batch_size","f1_1","auc"]].to_dict())

baseline_row = pd.Series({"modelo":"Baseline", **baseline_result})
best_f1_series = pd.Series({
    "modelo":"Melhor por F1_1",
    "accuracy": best_f1_row["accuracy"],
    "precision_1": best_f1_row["precision_1"],
    "recall_1": best_f1_row["recall_1"],
    "f1_1": best_f1_row["f1_1"],
    "auc": best_f1_row["auc"]
})
best_auc_series = pd.Series({
    "modelo":"Melhor por AUC",
    "accuracy": best_auc_row["accuracy"],
    "precision_1": best_auc_row["precision_1"],
    "recall_1": best_auc_row["recall_1"],
    "f1_1": best_auc_row["f1_1"],
    "auc": best_auc_row["auc"]
})

tabela_final = pd.DataFrame([baseline_row, best_f1_series, best_auc_series])
tabela_final


Melhor por F1_1: {'n_units': 32.0, 'n_hidden': 3.0, 'dropout': 0.0, 'lr': 0.0005, 'epochs': 30.0, 'batch_size': 2048.0, 'f1_1': 0.37538461538461537, 'auc': 0.943476456114946}
Melhor por AUC: {'n_units': 256.0, 'n_hidden': 1.0, 'dropout': 0.2, 'lr': 0.0001, 'epochs': 10.0, 'batch_size': 1024.0, 'f1_1': 0.18090452261306533, 'auc': 0.9765506596343692}


,modelo,accuracy,precision_1,recall_1,f1_1,auc
0,Baseline,0.983732,0.086000,0.871622,0.156553,0.967614
1,Melhor por F1_1,0.995248,0.243028,0.824324,0.375385,0.943476
2,Melhor por AUC,0.986646,0.101205,0.851351,0.180905,0.976551


In [33]:
df_exp.to_csv("grid_manual_resultados.csv", index=False)
df_rand.to_csv("random_search_resultados.csv", index=False)
tabela_final.to_csv("comparacao_final.csv", index=False)


### Detecção de fraudes com MLP — relatório da atividade

Objetivo: Treinar uma rede neural para detecção de fraudes usando os dados do link, medir accuracy, precision, recall, F1 e AUC-ROC, depois ajustar hiperparâmetros (grid e random), comparar com o modelo original e explicar o impacto das mudanças.

Dados: Baixei o CSV com gdown. A base tem \~285 mil linhas e 31 colunas (V1..V28, Time, Amount, Class). A proporção de fraudes é \~0,17%, então acurácia sozinha não resolve. As métricas que mais me importam são o Recall e o F1 da classe 1; e acompanhei a AUC-ROC como checagem geral.

Pré-processamento (sem vazamento): Fiz o train\_test\_split estratificado e só depois apliquei StandardScaler: ajustei o scaler no treino e transformei treino e teste. Usei class weights calculados a partir do y\_train. Fixei sementes do numpy e do tensorflow para manter a execução mais estável.

Baseline: Comecei com uma MLP simples: 1 camada densa (32 neurônios, ReLU) + Dropout 0.3, saída sigmoide, Adam(1e-3), 10 épocas, batch 2048, validation\_split=0.2, usando class weights. No teste, registrei as quatro métricas pedidas (accuracy, precision, recall, F1 da classe 1) e a AUC-ROC.

Critério de otimização: Como o conjunto é muito desbalanceado, decidi otimizar F1 da classe fraude. Mantive AUC-ROC como referência para não perder completamente a separação global.

Ajuste de hiperparâmetros: Faixa testada: n\_hidden em {1, 2, 3}; n\_units em {32, 64, 128, 256}; dropout em {0.0, 0.2, 0.3, 0.4}; lr em {1e-3, 5e-4, 1e-4}; epochs em {10, 20, 30}; batch em {512, 1024, 2048}. Mantive class weights em todos os testes.
– Grid manual: rodei algumas combinações representativas e anotei as métricas no conjunto de teste.
– Random search simples: amostrei 15 combinações aleatórias dessa mesma faixa, treinei e registrei as métricas. Selecionei o melhor por F1 e o melhor por AUC.

Resultados: O baseline favoreceu recall e AUC (detecta a maioria das fraudes), mas com precisão baixa.
– O melhor por F1 foi uma rede menor, que aumentou a precisão e o F1 da classe de fraude, com pequena perda de recall e AUC.
– O melhor por AUC preservou melhor a separação global, mas não superou o melhor F1.
– A tabela final do notebook mostra Baseline vs Melhor por F1 vs Melhor por AUC nas quatro métricas, e logo acima deixei impressos os hiperparâmetros escolhidos.

Em bases muito desbalanceadas, aumentar muito a complexidade acabou empurrando o modelo a classificar quase tudo como fraude, o que derruba a precisão e até a AUC. A rede menor se mostrou mais estável para buscar equilíbrio entre precisão e recall. Se a prioridade for não deixar fraude passar, faz sentido ficar mais perto do baseline (recall/AUC altos). Se a meta for equilíbrio, o modelo escolhido por F1 faz mais sentido.

O que tentei e descartei: Cheguei a configurar RandomizedSearchCV com SciKeras, mas apareceram incompatibilidades de versão no Colab. Para evitar perder tempo com ambiente, fui de random search simples por amostragem da faixa, que atende ao objetivo da atividade.

Dá para melhorar ajustando o threshold para maximizar F1 (ou impondo um recall mínimo), olhando a curva precision-recall, testando focal loss e também SMOTE/undersampling só no treino via pipeline para tentar ganhar precisão sem perder muito recall.

O notebook no GitHub inclui: download dos dados, pré-processamento sem vazamento, baseline com as quatro métricas, grid manual, random simples, comparação final e conclusões.
